In [ ]:
%matplotlib inline

%load_ext autoreload
%autoreload 2

## Std libs
from collections import defaultdict
from datetime import datetime

## Non-std libs
import matplotlib.pyplot as plt

## Local modules
from scn_rrd import rrd_meta_utils, rrd_utils, plot_utils

### Count users by counting UEs

In [ ]:
class At_eNB:
    def __init__(self):
        self.locations = [
            'FCS',
            'sps-franklin',
        ]
        
        meta = rrd_meta_utils.read_meta()
        self.meta = meta[
            ( meta['location'].isin(self.locations) ) &
            ( meta['device_group_name'] == 'eNB' )
        ]
        devices = self.meta[['hostname', 'location']].drop_duplicates()
        self.hostname_to_meta = {
            row['hostname'] : row
            for (_, row) in devices.iterrows()
        }
    
    def read_rrds(self):
        self.hostname_to_df = rrd_utils.read_rrds(
            self.hostname_to_meta.keys(),
            rrd_meta_utils.ENB_DEVICE_RRD_FILENAMES.clients,
            '-2month'
        )
        for df in self.hostname_to_df.values():
            df.rename(columns={'sensor': 'users_ct'}, inplace=True)

    def format_label(self, hostname):
        loc = self.hostname_to_meta[hostname]['location']
        return f"{loc} LTE"

In [ ]:
at_enb = At_eNB()

In [ ]:
at_enb.read_rrds()
# Right now, BaicellsNova436Q devices do not have wireless-sensor-clients-rts-1.rrd.
# If we see stderr here, this is expected.

### Count users by DHCP

In [ ]:
class At_DHCP:
    def __init__(self):
        self.locations = [
            'lihi-southend',
            'nickelsville-cd',
            'northlake',
        ]
        
        meta = rrd_meta_utils.read_meta()
        self.meta = meta[
            ( meta['location'].isin(self.locations) ) &
            ( meta['device_group_name'].isin(['CPE gateway', 'Backhaul gateway']) )
        ]
        devices = self.meta[['hostname', 'location', 'device_group_name']].drop_duplicates()
        self.hostname_to_meta = {
            row['hostname'] : row
            for (_, row) in devices.iterrows()
        }
    
    def read_rrds(self):
        self.hostname_to_df = rrd_utils.read_rrds(
            self.hostname_to_meta.keys(),
            rrd_meta_utils.MIKROTIK_DEVICE_RRD_FILENAMES.clients,
            '-2month'
        )
        for df in self.hostname_to_df.values():
            df.rename(columns={'leases': 'users_ct'}, inplace=True)
    
    def format_label(self, hostname):
        (loc, devGrpName) = self.hostname_to_meta[hostname][['location', 'device_group_name']]
        if devGrpName == 'CPE gateway':
            user_type = 'LTE'
        else:
            user_type = 'WiFi'
        return f"{loc} {user_type}"

In [ ]:
at_dhcp = At_DHCP()

In [ ]:
at_dhcp.read_rrds()

### Plot

In [ ]:
phys_loc_to_ct = defaultdict(int)
for namespace in [at_enb, at_dhcp]:
    for (hostname, df) in namespace.hostname_to_df.items():
        phys_loc = namespace.hostname_to_meta[hostname]['location']
        phys_loc_to_ct[phys_loc] += 1

phys_loc_to_remaining_ct = dict(phys_loc_to_ct)
        
(fig, ax) = plt.subplots(figsize=(12, 6))
for namespace in [at_enb, at_dhcp]:
    for (hostname, df) in namespace.hostname_to_df.items():
        phys_loc = namespace.hostname_to_meta[hostname]['location']
        color = plot_utils.LOC_TO_COLOR[phys_loc]
        
        j = phys_loc_to_remaining_ct[phys_loc]
        phys_loc_to_remaining_ct[phys_loc] = j-1
        color_s = j / phys_loc_to_ct[phys_loc]
        
        label = namespace.format_label(hostname)
        
        ax.plot(df['time'], df['users_ct'], label=label, color=color)
ax.set_title('Count of users')
ax.tick_params(axis='x', labelrotation=90)
ax.set_xlim(left=datetime(2022,12,22), right=datetime.now())
ax.legend()
None # Hide stdout output of the above line